# 🚀 Dewey RAG System - Server Setup & Troubleshooting

**AMD MI300X GPU Server Setup Guide**

This notebook helps you set up the Dewey Decimal Classification RAG system on your high-performance AMD MI300X server, troubleshooting common issues with Python 3.13 compatibility and GPU support.

## Server Specifications
- **GPU**: AMD MI300X (192GB VRAM)  
- **RAM**: 240GB
- **CPU**: 20 vCPUs
- **Python**: 3.13.x
- **System**: ROCm-enabled environment

---

## 🔍 Section 1: Check Python Version and Environment

Let's first examine your current Python environment to understand the compatibility issues.

In [ ]:
import sys
import platform
import subprocess
import pkg_resources

print("🐍 Python Environment Information")
print("=" * 40)
print(f"Python Version: {sys.version}")
print(f"Python Executable: {sys.executable}")
print(f"Platform: {platform.platform()}")
print(f"Architecture: {platform.machine()}")

# Check pip version
try:
    pip_version = subprocess.run([sys.executable, "-m", "pip", "--version"], 
                                capture_output=True, text=True)
    print(f"Pip Version: {pip_version.stdout.strip()}")
except Exception as e:
    print(f"Pip check failed: {e}")

# Check if we're in a virtual environment
print(f"Virtual Environment: {sys.prefix != sys.base_prefix}")
print(f"Base Prefix: {sys.base_prefix}")
print(f"Current Prefix: {sys.prefix}")

# Check Python version compatibility
major, minor = sys.version_info[:2]
print(f"\n📊 Python Version Analysis:")
print(f"Major.Minor: {major}.{minor}")
if (major, minor) >= (3, 13):
    print("⚠️  You're using Python 3.13+ - many ML packages may not be compatible yet")
    print("   Consider using Python 3.11 or 3.12 for better compatibility")
else:
    print("✅ Python version should be compatible with most ML packages")

## 🔧 Section 2: Analyze pip Installation Errors

Let's analyze the specific error you encountered and understand why certain packages failed to install.

In [ ]:
print("🔍 Installation Error Analysis")
print("=" * 30)

# The main issues from your error log:
issues = [
    {
        "package": "faiss-gpu",
        "error": "No matching distribution found",
        "reason": "faiss-gpu is not available for all platforms/Python versions",
        "solution": "Use faiss-cpu or install via conda"
    },
    {
        "package": "Various packages", 
        "error": "Requires Python >=3.8,<3.12",
        "reason": "Many ML packages don't support Python 3.13 yet",
        "solution": "Use compatible versions or downgrade Python"
    }
]

for i, issue in enumerate(issues, 1):
    print(f"\n🚨 Issue {i}: {issue['package']}")
    print(f"   Error: {issue['error']}")
    print(f"   Reason: {issue['reason']}")
    print(f"   Solution: {issue['solution']}")

print(f"\n💡 Recommendation:")
print("   Use the Python 3.13 compatible requirements file we'll create")
print("   Install faiss-cpu instead of faiss-gpu (performance difference is minimal)")

# Check if faiss-gpu is available
try:
    result = subprocess.run([sys.executable, "-m", "pip", "index", "versions", "faiss-gpu"], 
                          capture_output=True, text=True)
    print(f"\n🔍 faiss-gpu availability check:")
    print(result.stdout if result.stdout else "No versions found")
    if result.stderr:
        print(f"Error: {result.stderr}")
except Exception as e:
    print(f"Could not check faiss-gpu availability: {e}")

## 📦 Section 3: Install Python 3.13 Compatible Packages

Let's install packages that are compatible with Python 3.13, using alternative approaches where needed.

In [ ]:
# Install core packages first (these should work with Python 3.13)
core_packages = [
    "torch>=2.4.0",
    "torchvision>=0.19.0",
    "torchaudio>=2.4.0", 
    "transformers>=4.40.0",
    "numpy>=1.26.0",
    "pandas>=2.2.0",
    "scipy>=1.12.0"
]

print("🚀 Installing Core ML Packages")
print("=" * 35)

for package in core_packages:
    print(f"Installing {package}...")
    try:
        result = subprocess.run([sys.executable, "-m", "pip", "install", package], 
                              capture_output=True, text=True, timeout=300)
        if result.returncode == 0:
            print(f"✅ {package} installed successfully")
        else:
            print(f"❌ {package} installation failed:")
            print(f"   {result.stderr}")
    except subprocess.TimeoutExpired:
        print(f"⏱️ {package} installation timed out (this is normal for large packages)")
    except Exception as e:
        print(f"❌ Error installing {package}: {e}")

print("\n📊 Installation Summary Complete")

In [ ]:
# Install FAISS-CPU (alternative to faiss-gpu)
print("🔍 Installing FAISS and Vector Search Libraries")
print("=" * 45)

faiss_packages = [
    "faiss-cpu>=1.8.0",  # CPU version, more compatible
    "sentence-transformers>=2.7.0",
    "scikit-learn>=1.4.0"
]

for package in faiss_packages:
    print(f"Installing {package}...")
    try:
        result = subprocess.run([sys.executable, "-m", "pip", "install", package], 
                              capture_output=True, text=True, timeout=300)
        if result.returncode == 0:
            print(f"✅ {package} installed successfully")
        else:
            print(f"❌ {package} installation failed:")
            print(f"   Error: {result.stderr[:200]}...")
    except Exception as e:
        print(f"❌ Error installing {package}: {e}")

# Alternative: Try installing via conda if pip fails
print(f"\n💡 Note: If faiss-cpu installation fails via pip, you can try:")
print(f"   conda install -c conda-forge faiss-cpu")
print(f"   This often works better for FAISS installation")

## 🎯 Section 4: Handle Version Compatibility Issues

Let's address the specific compatibility issues and install remaining packages.

In [ ]:
# Install additional packages for the RAG system
additional_packages = [
    "PyPDF2>=3.0.0",
    "pdfplumber>=0.10.0", 
    "pymupdf>=1.24.0",
    "datasets>=2.19.0",
    "tiktoken>=0.6.0",
    "tqdm>=4.66.0",
    "streamlit>=1.33.0",
    "plotly>=5.20.0",
    "fastapi>=0.110.0",
    "uvicorn>=0.29.0"
]

print("📚 Installing Additional RAG System Packages")
print("=" * 45)

successful_installs = []
failed_installs = []

for package in additional_packages:
    print(f"Installing {package}...")
    try:
        result = subprocess.run([sys.executable, "-m", "pip", "install", package], 
                              capture_output=True, text=True, timeout=180)
        if result.returncode == 0:
            print(f"✅ {package}")
            successful_installs.append(package)
        else:
            print(f"❌ {package} - {result.stderr.split('ERROR:')[1].split('\\n')[0] if 'ERROR:' in result.stderr else 'Unknown error'}")
            failed_installs.append(package)
    except Exception as e:
        print(f"❌ {package} - {str(e)}")
        failed_installs.append(package)

print(f"\n📊 Installation Results:")
print(f"✅ Successful: {len(successful_installs)}")
print(f"❌ Failed: {len(failed_installs)}")

if failed_installs:
    print(f"\n🔧 Failed packages can be installed individually later:")
    for pkg in failed_installs:
        print(f"   pip install {pkg}")

## 🖥️ Section 5: Verify GPU Support and ROCm

Let's check your AMD MI300X GPU availability and ROCm installation for optimal performance.

In [ ]:
print("🖥️ GPU and ROCm Status Check")
print("=" * 30)

# Check if torch detects GPU
try:
    import torch
    print(f"✅ PyTorch imported successfully")
    print(f"PyTorch version: {torch.__version__}")
    
    # Check CUDA availability (might work for ROCm too)
    if torch.cuda.is_available():
        print(f"🚀 CUDA/ROCm available: {torch.cuda.is_available()}")
        print(f"Device count: {torch.cuda.device_count()}")
        
        if torch.cuda.device_count() > 0:
            print(f"Current device: {torch.cuda.current_device()}")
            print(f"Device name: {torch.cuda.get_device_name(0)}")
            
            # Check memory
            if hasattr(torch.cuda, 'get_device_properties'):
                props = torch.cuda.get_device_properties(0)
                print(f"GPU Memory: {props.total_memory / 1024**3:.1f} GB")
            
            # Test tensor creation on GPU
            try:
                test_tensor = torch.randn(10, 10).cuda()
                print("✅ GPU tensor creation successful")
                del test_tensor
                torch.cuda.empty_cache()
            except Exception as e:
                print(f"❌ GPU tensor creation failed: {e}")
    else:
        print("⚠️ CUDA/ROCm not detected by PyTorch")
        print("   The system will use CPU (still functional but slower)")
        
except ImportError:
    print("❌ PyTorch not installed or not importable")

# Check ROCm installation
print(f"\n🔍 Checking ROCm installation...")
try:
    result = subprocess.run(["rocm-smi"], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ ROCm tools available")
        print("ROCm GPU Status:")
        print(result.stdout[:500] + "..." if len(result.stdout) > 500 else result.stdout)
    else:
        print("⚠️ ROCm tools not found or not working")
except Exception as e:
    print(f"⚠️ Could not check ROCm: {e}")

print(f"\n💡 Note: Even without GPU acceleration, the RAG system will work on CPU")
print(f"   Performance will be slower but still functional for testing")

## ✅ Section 6: Test Package Imports

Let's verify that all critical packages can be imported and are working correctly.

In [ ]:
print("🧪 Testing Package Imports")
print("=" * 28)

# Critical packages for the RAG system
test_packages = {
    'torch': 'PyTorch for deep learning',
    'transformers': 'Hugging Face transformers',
    'numpy': 'Numerical computing',
    'pandas': 'Data manipulation',
    'faiss': 'Vector similarity search',
    'sentence_transformers': 'Sentence embeddings',
    'PyPDF2': 'PDF processing',
    'pdfplumber': 'Advanced PDF extraction',
    'streamlit': 'Web interface',
    'plotly': 'Interactive visualizations'
}

successful_imports = []
failed_imports = []

for package, description in test_packages.items():
    try:
        __import__(package)
        print(f"✅ {package:20} - {description}")
        successful_imports.append(package)
    except ImportError as e:
        print(f"❌ {package:20} - Import failed: {str(e)[:50]}...")
        failed_imports.append(package)

print(f"\n📊 Import Test Results:")
print(f"✅ Working: {len(successful_imports)}/{len(test_packages)}")
print(f"❌ Failed: {len(failed_imports)}")

if failed_imports:
    print(f"\n🔧 Failed imports:")
    for pkg in failed_imports:
        print(f"   To fix: pip install {pkg}")

# Test basic functionality
print(f"\n🚀 Testing Basic Functionality:")

try:
    import numpy as np
    import torch
    
    # Test numpy
    arr = np.random.randn(5, 5)
    print(f"✅ NumPy array creation: {arr.shape}")
    
    # Test torch
    tensor = torch.randn(5, 5)
    print(f"✅ PyTorch tensor creation: {tensor.shape}")
    
    # Test torch device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"✅ PyTorch device: {device}")
    
    if 'faiss' in successful_imports:
        import faiss
        # Test FAISS
        d = 64
        nb = 1000
        np.random.seed(1234)
        xb = np.random.random((nb, d)).astype('float32')
        index = faiss.IndexFlatL2(d)
        index.add(xb)
        print(f"✅ FAISS index creation: {index.ntotal} vectors")
    
    print(f"\n🎉 Core functionality tests passed!")
    
except Exception as e:
    print(f"❌ Functionality test failed: {e}")

print(f"\n🚀 System is ready for Dewey RAG setup!")

## 🚀 Section 7: Quick Dewey RAG System Setup

Now that packages are installed, let's quickly set up and test the RAG system.

In [ ]:
# Quick check of Dewey system files
import os
from pathlib import Path

print("📚 Dewey RAG System File Check")
print("=" * 32)

required_files = [
    "dewey-decimal-classification-ddc23-complete-1-4-23nbsped-1910608815-9781910608814_compress.pdf",
    "process_pdf_server.py", 
    "rag_system_server.py",
    "streamlit_gui.py",
    "deploy_server.py"
]

missing_files = []
for file in required_files:
    if Path(file).exists():
        size = Path(file).stat().st_size / (1024*1024)  # MB
        print(f"✅ {file:<60} ({size:.1f} MB)")
    else:
        print(f"❌ {file:<60} (missing)")
        missing_files.append(file)

if missing_files:
    print(f"\n⚠️ Missing files: {len(missing_files)}")
    print("   Please ensure all RAG system files are present")
else:
    print(f"\n🎉 All required files present!")

# Check if data directories exist
data_dirs = ["processed_data", "vector_db", "models_cache"]
print(f"\nChecking data directories:")
for dir_name in data_dirs:
    if Path(dir_name).exists():
        print(f"✅ {dir_name}/ exists")
    else:
        print(f"📁 {dir_name}/ will be created during setup")

print(f"\n🚀 Ready to run: python deploy_server.py --step full")

In [ ]:
# Test RAG system component imports
print("🧪 Testing RAG System Imports")
print("=" * 30)

components_to_test = [
    ('torch', 'PyTorch (GPU processing)'),
    ('transformers', 'HuggingFace Transformers'),
    ('faiss', 'Vector database (FAISS)'),
    ('numpy', 'NumPy (numerical ops)'),
    ('pandas', 'Data handling'),
    ('streamlit', 'Web interface'),
    ('fastapi', 'API server'),
    ('uvicorn', 'ASGI server'),
    ('pdfplumber', 'PDF processing'),
    ('tqdm', 'Progress bars'),
    ('plotly', 'Interactive plots'),
    ('matplotlib', 'Static plots'),
    ('seaborn', 'Statistics plots')
]

import_results = {}
for module, description in components_to_test:
    try:
        __import__(module)
        print(f"✅ {module:<15} - {description}")
        import_results[module] = True
    except ImportError as e:
        print(f"❌ {module:<15} - {description} (Error: {str(e)[:50]}...)")
        import_results[module] = False
    except Exception as e:
        print(f"⚠️ {module:<15} - {description} (Warning: {str(e)[:50]}...)")
        import_results[module] = False

# Summary
successful = sum(import_results.values())
total = len(import_results)
print(f"\n📊 Import Success Rate: {successful}/{total} ({successful/total*100:.1f}%)")

if successful == total:
    print("🎉 All components available! RAG system ready.")
elif successful >= total * 0.8:
    print("⚠️ Most components available. Check specific failures above.")
else:
    print("❌ Multiple import failures. Review package installation.")

## 🎯 Section 8: Test Basic RAG Functionality

If all imports succeed, test a minimal RAG workflow to ensure everything works together.

In [ ]:
# Test basic RAG system workflow (if imports successful)
try:
    print("🔬 Testing Basic RAG Components")
    print("=" * 32)
    
    # Test PyTorch + GPU
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")
    print(f"✅ CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"✅ GPU devices: {torch.cuda.device_count()}")
        print(f"✅ Current device: {torch.cuda.get_device_name()}")
    
    # Test Transformers
    from transformers import __version__ as tf_version
    print(f"✅ Transformers version: {tf_version}")
    
    # Test FAISS (vector database)
    import faiss
    print(f"✅ FAISS version: {faiss.__version__}")
    
    # Test basic embeddings (small test)
    import numpy as np
    dimension = 64  # Small test
    index = faiss.IndexFlatL2(dimension)
    test_vectors = np.random.random((10, dimension)).astype('float32')
    index.add(test_vectors)
    
    # Search test
    query_vector = np.random.random((1, dimension)).astype('float32')
    distances, indices = index.search(query_vector, k=3)
    print(f"✅ FAISS vector search test: Found {len(indices[0])} results")
    
    print("\n🎉 Basic RAG components working!")
    print("🚀 Ready to run full RAG system!")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("📝 Please install missing packages first")
except Exception as e:
    print(f"⚠️ Test error: {e}")
    print("🔍 Components imported but functionality test failed")

## ⚠️ CRITICAL: Python Version Compatibility Issue

**Problem**: Python 3.13 is too new! Many ML packages don't support it yet.

**From your error log**, I can see:
- `faiss-gpu>=1.7.4` has NO versions for Python 3.13  
- Other packages show: `Requires-Python >=3.9,<3.13` (excludes 3.13)

**Recommended Python Versions for ML/AI**:
- **Python 3.11** ✅ (Best choice - most stable, widely supported)
- **Python 3.10** ✅ (Also excellent, very stable)  
- Python 3.12 ⚠️ (Newer, some packages may be limited)
- Python 3.13 ❌ (Too new, many packages not available)

**Solution**: Create new conda environment with Python 3.11

In [ ]:
# SOLUTION: Create Python 3.11 environment
print("🔧 Setting up Python 3.11 environment for ML/AI packages")
print("=" * 55)

print("📋 Commands to run:")
print()
print("# 1. Remove current environment (if it exists)")
print("conda deactivate")  
print("conda env remove --name dewey-rag-server")
print()
print("# 2. Create new environment with Python 3.11")
print("conda create --name dewey-rag-server python=3.11 -y")
print()
print("# 3. Activate new environment") 
print("conda activate dewey-rag-server")
print()
print("# 4. Install packages (choose ONE option):")
print()
print("Option A - Original requirements:")
print("pip install -r requirements_server.txt")
print()
print("Option B - Python 3.11 optimized requirements:")
print("pip install -r requirements_server_py311.txt")
print()
print("🎯 After setup, all packages should install successfully!")

# Let's also check what the current environment file specifies
try:
    with open('environment_server.yml', 'r') as f:
        content = f.read()
        if 'python=3.13' in content:
            print(f"\n⚠️ IMPORTANT: Your environment_server.yml specifies python=3.13")
            print(f"   You should update it to python=3.11 for better compatibility")
        elif 'python=3.11' in content:
            print(f"\n✅ Your environment_server.yml already specifies python=3.11") 
        else:
            print(f"\n📝 Check your environment_server.yml for Python version")
except FileNotFoundError:
    print(f"\n📄 No environment_server.yml found")

## 🎉 Final Setup Instructions

**Your environment_server.yml already uses Python 3.10** - that's perfect!

Choose one of these approaches:

In [1]:
# 🎯 RECOMMENDED SETUP COMMANDS
print("🚀 Complete Setup Commands for AMD MI300X Server")
print("=" * 50)

print("💡 Your environment_server.yml uses Python 3.10 - Perfect!")
print()

print("📋 OPTION 1: Use Conda Environment (Recommended)")
print("-" * 45)
print("conda deactivate")
print("conda env remove --name dewey-rag-server -y")  
print("conda env create -f environment_server.yml")
print("conda activate dewey-rag-server")
print()

print("📋 OPTION 2: Manual Python 3.11 Setup")  
print("-" * 37)
print("conda deactivate")
print("conda env remove --name dewey-rag-server -y")
print("conda create --name dewey-rag-server python=3.11 -y")
print("conda activate dewey-rag-server") 
print("pip install -r requirements_server_py311.txt")
print()

print("🎯 After environment setup:")
print("python deploy_server.py --step full")
print()

print("🔥 Launch interfaces:")
print("streamlit run streamlit_gui.py          # Web GUI")  
print("python cli_interface.py --help         # CLI tool")
print("python launcher.py                     # Unified launcher")

print()
print("✨ Pro tip: The conda environment approach (Option 1) is most reliable!")
print("   It handles GPU drivers and dependencies automatically.")

🚀 Complete Setup Commands for AMD MI300X Server
💡 Your environment_server.yml uses Python 3.10 - Perfect!

📋 OPTION 1: Use Conda Environment (Recommended)
---------------------------------------------
conda deactivate
conda env remove --name dewey-rag-server -y
conda env create -f environment_server.yml
conda activate dewey-rag-server

📋 OPTION 2: Manual Python 3.11 Setup
-------------------------------------
conda deactivate
conda env remove --name dewey-rag-server -y
conda create --name dewey-rag-server python=3.11 -y
conda activate dewey-rag-server
pip install -r requirements_server_py311.txt

🎯 After environment setup:
python deploy_server.py --step full

🔥 Launch interfaces:
streamlit run streamlit_gui.py          # Web GUI
python cli_interface.py --help         # CLI tool
python launcher.py                     # Unified launcher

✨ Pro tip: The conda environment approach (Option 1) is most reliable!
   It handles GPU drivers and dependencies automatically.


## 🎉 ChromaDB Integration Added!

**Great choice!** ChromaDB is often better than FAISS for many use cases:
- ✅ **Easier to use** - No manual index management
- ✅ **Built-in persistence** - Automatic data storage
- ✅ **Better metadata support** - Rich querying capabilities  
- ✅ **Modern architecture** - Active development and features
- ✅ **Python-native** - Better integration with ML workflows

**New files created**:
- `rag_system_chromadb.py` - Complete ChromaDB-based RAG system
- Updated `requirements_server_py311.txt` - Now includes ChromaDB

In [2]:
# 🧪 Test ChromaDB Setup
print("🔬 Testing ChromaDB Installation and Basic Functionality")
print("=" * 55)

try:
    # Test ChromaDB import
    import chromadb
    print(f"✅ ChromaDB version: {chromadb.__version__}")
    
    # Test basic ChromaDB functionality
    from chromadb.config import Settings
    
    # Create a temporary in-memory client for testing
    client = chromadb.EphemeralClient(settings=Settings(anonymized_telemetry=False))
    
    # Create test collection
    collection = client.create_collection(
        name="test_collection",
        metadata={"description": "Test collection"}
    )
    
    # Add test data
    test_docs = [
        "ChromaDB is a vector database for AI applications",
        "It provides efficient similarity search capabilities",
        "Vector databases are essential for RAG systems"
    ]
    
    collection.add(
        documents=test_docs,
        ids=[f"doc_{i}" for i in range(len(test_docs))],
        metadatas=[{"source": f"test_{i}"} for i in range(len(test_docs))]
    )
    
    # Test search
    results = collection.query(
        query_texts=["What is ChromaDB?"],
        n_results=2
    )
    
    print(f"✅ ChromaDB basic functionality working!")
    print(f"   - Collection created: {collection.name}")
    print(f"   - Documents added: {collection.count()}")
    print(f"   - Search results: {len(results['documents'][0])}")
    print(f"   - Top result: {results['documents'][0][0][:50]}...")
    
except ImportError as e:
    print(f"❌ ChromaDB not installed: {e}")
    print("   Install with: pip install chromadb")
except Exception as e:
    print(f"⚠️ ChromaDB test failed: {e}")

print(f"\n🚀 If ChromaDB works, you can use:")
print(f"   python rag_system_chromadb.py  # ChromaDB-based RAG system")
print(f"   python rag_system_server.py    # Original FAISS-based system")

🔬 Testing ChromaDB Installation and Basic Functionality
❌ ChromaDB not installed: No module named 'chromadb'
   Install with: pip install chromadb

🚀 If ChromaDB works, you can use:
   python rag_system_chromadb.py  # ChromaDB-based RAG system
   python rag_system_server.py    # Original FAISS-based system


## 📊 FAISS vs ChromaDB Comparison

| Feature | FAISS | ChromaDB | Winner |
|---------|-------|----------|---------|
| **Setup Complexity** | Manual index management | Automatic persistence | 🏆 ChromaDB |
| **Metadata Support** | Limited, manual tracking | Rich metadata querying | 🏆 ChromaDB |
| **Performance** | Extremely fast, optimized | Fast, good for most cases | 🏆 FAISS |
| **Memory Usage** | Lower memory footprint | Moderate memory usage | 🏆 FAISS |
| **Scalability** | Handles millions of vectors | Good for medium scale | 🏆 FAISS |
| **Ease of Use** | Complex API, manual work | Simple, intuitive API | 🏆 ChromaDB |
| **GPU Support** | Excellent GPU acceleration | CPU-focused | 🏆 FAISS |
| **Development** | Facebook/Meta (stable) | Active, modern features | 🏆 ChromaDB |
| **Integration** | Manual embedding pipeline | Built-in embedding options | 🏆 ChromaDB |

**Recommendation**: 
- **Use ChromaDB** 🌟 - Better for development, easier to use, modern features
- **Use FAISS** 🚀 - Only if you need maximum performance or >1M vectors

In [3]:
# 🎯 Updated Commands with ChromaDB Support
print("🚀 Dewey RAG System - Now with ChromaDB!")
print("=" * 40)

print("📋 Setup Commands (Python 3.10/3.11):")
print("conda env create -f environment_server.yml")
print("conda activate dewey-rag-server")
print()

print("🎯 Deployment Options:")
print()
print("Option 1: ChromaDB (Recommended) 🌟")
print("python deploy_unified.py --vector-db chromadb --step full")
print()
print("Option 2: FAISS (High Performance) 🚀")  
print("python deploy_unified.py --vector-db faiss --step full")
print()

print("🔥 Launch Applications:")
print("streamlit run streamlit_gui.py      # Web interface")
print("python cli_interface.py             # Command line")  
print("python launcher.py                  # Unified launcher")
print()

print("💡 Pro Tips:")
print("• ChromaDB: Easier to use, better for development")
print("• FAISS: Maximum speed, best for large datasets")
print("• Both support your AMD MI300X GPU for model inference")
print("• ChromaDB stores data automatically (no manual saving)")
print("• FAISS gives you more control over indexing parameters")

print(f"\n✨ Your choice of ChromaDB is excellent for getting started!")
print(f"   You can always switch to FAISS later if needed.")

🚀 Dewey RAG System - Now with ChromaDB!
📋 Setup Commands (Python 3.10/3.11):
conda env create -f environment_server.yml
conda activate dewey-rag-server

🎯 Deployment Options:

Option 1: ChromaDB (Recommended) 🌟
python deploy_unified.py --vector-db chromadb --step full

Option 2: FAISS (High Performance) 🚀
python deploy_unified.py --vector-db faiss --step full

🔥 Launch Applications:
streamlit run streamlit_gui.py      # Web interface
python cli_interface.py             # Command line
python launcher.py                  # Unified launcher

💡 Pro Tips:
• ChromaDB: Easier to use, better for development
• FAISS: Maximum speed, best for large datasets
• Both support your AMD MI300X GPU for model inference
• ChromaDB stores data automatically (no manual saving)
• FAISS gives you more control over indexing parameters

✨ Your choice of ChromaDB is excellent for getting started!
   You can always switch to FAISS later if needed.
